<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Analyzing%20audio/Querying_RDBMS_via_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sounddevice
!pip install scipy
!pip install playsound
!apt-get install -y libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

!apt install ffmpeg
!apt-get install libportaudio2

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-e78uddqz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-e78uddqz
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import argparse
import openai
import pandas as pd
import time
from IPython.display import Markdown, display, Image, Audio
from google.colab import files
import base64
import requests
import shutil
import scipy.io.wavfile
import sounddevice
import sqlite3
import time
import os
import numpy as np
import re

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

from google.colab import drive, userdata,  output
from openai import OpenAI
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    API_KEY = userdata.get('OPENAI_API_KEY')
    if not API_KEY:
        raise ValueError("API key not found. Please add it to Colab Secrets.")
except ImportError:
    # Fallback for environments other than Colab, though not recommended
    API_KEY = os.environ.get("OPENAI_API_KEY")
    if not API_KEY:
        raise ValueError("API_KEY environment variable not set.")

API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key= API_KEY)

In [ ]:
def get_structure(data_path):
    """ Extract structure from SQLite database.

    Args:
        data_path: path to SQLite data file.

    Returns:
        text description of database structure.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute("select sql from sqlite_master where type = 'table';")
        table_rows = cursor.fetchall()
        table_ddls = [r[0] for r in table_rows]
        return '\n'.join(table_ddls)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
"""
The above recording goes to the Data/audio_data/my_recording2.wav

"""

file_name = 'my_recording2.wav'

drive_directory = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/'
drive_path = os.path.join(drive_directory, file_name)

print(f"Ensuring directory exists: {drive_directory}")
os.makedirs(drive_directory, exist_ok=True)

with open('recording.webm', 'wb') as f:
  f.write(recorder.audio.value)

print(f"Converting to .wav format: {file_name}")
!ffmpeg -i recording.webm -ac 1 -f wav {file_name} -y -hide_banner -loglevel panic

print(f"Copying file to Google Drive: {drive_path}")
!cp {file_name} "{drive_path}"

os.remove('recording.webm')
os.remove(file_name)

print("\n✅ Recording successfully saved to your Google Drive!")
print(f"File path: {drive_path}")

Ensuring directory exists: /content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/
Converting to .wav format: my_recording2.wav
Copying file to Google Drive: /content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/my_recording2.wav

✅ Recording successfully saved to your Google Drive!
File path: /content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/my_recording2.wav


In [ ]:
def transcribe(audio_path):
    """ Transcribe audio file to text.

    Args:
        audio_path: path to audio file.

    Returns:
        transcribed text.
    """
    with open(audio_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file, model='whisper-1')
        return transcription.text


In [ ]:
def create_prompt(description, question):
    """ Generate prompt to translate question into SQL query.

    Args:
        description: text description of database structure.
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += [description]
    parts += ['Translate this question into SQL query:']
    parts += [question]
    parts += ['SQL Query:']
    return '\n'.join(parts)

In [ ]:
def call_llm(prompt):
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {'role':'user', 'content':prompt}
                    ]
                )
            return response.choices[0].message.content
        except:
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query OpenAI model!')

In [ ]:
def process_query(data_path, query):
    """ Processes SQL query and returns result.

    Args:
        data_path: path to SQLite data file.
        query: process this query on database.

    Returns:
        query result.
    """
    with sqlite3.connect(data_path) as connection:
        cursor = connection.cursor()
        cursor.execute(query)
        table_rows = cursor.fetchall()
        table_strings = [str(r) for r in table_rows]
        return '\n'.join(table_strings)

In [ ]:
db_path = '/content/drive/MyDrive/Colab Notebooks/DataScience+GPT/Data/structured_data/games.db'

data_structure = get_structure(db_path)
print(data_structure)


audio_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/audio_data/my_recording2.wav'

question = transcribe(audio_path)
print(f'Question: {question}')


prompt = create_prompt(data_structure, question)
print(prompt)

answer = call_llm(prompt)
query = re.findall('```sql(.*)```', answer, re.DOTALL)[0]
print(f'SQL: {query}')

try:
  result = process_query(db_path, query)
  print(f'Result: {result}')
except:
  print('Error processing query! Try to reformulate.')



CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Question: Which games sold more copies in Europe than Japan? So give me the top 10 games.
Database:
CREATE TABLE games(rank int, name text, platform text, year int, genre text, publisher text, americasales numeric, eusales numeric, japansales numeric, othersales numeric, globalsales numeric)
Translate this question into SQL query:
Which games sold more copies in Europe than Japan? So give me the top 10 games.
SQL Query:
SQL: 
SELECT rank, name, platform, year, genre, publisher, americasales, eusales, japansales, othersales, globalsales
FROM games
WHERE eusales > japansales
ORDER BY eusales DESC
LIMIT 10;

Result: (1, 'Wii Sports', 'Wii', 2006, 'Sports', 'Nintendo', 41.49, 29.02, 3.77, 8.46, 82.74)
(3, 'Mario Kart Wii', 'Wii', 2008, 'Racing', 'Nintendo', 15.85, 12.88, 3.79, 3.31, 35.82)
(4, 'Wii S